<a href="https://colab.research.google.com/gist/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e/untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cough classification notebook

In [1]:
# 安装 ffmpeg

# apt update
# apt install ffmpeg
# ffmpeg -version

## Importing Libraries

In [2]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# #Keras
# import keras

import warnings
warnings.filterwarnings('ignore')

%cd ../../


D:\github_projects\cough_videos


## Extracting cough audios and features

### Dataset

We use [COUGHVID](https://zenodo.org/record/7024894#.YysPHHZBybg) dataset for classification. 
<br>
<br>
数据原本将每段咳嗽音区分为 i.e
 * COVID-19
 * healthy
 * symptomatic
 
Total dataset: xxx samples


我们现在将任务改为二分类：healthy (label 0) vs unhealthy (label 1)

## Meta data


In [3]:
# # 查看每个json文件，选取标注

# import json

# dict_id2label = {}

# folder = './datasets/coughvid_v1/public_dataset'
# count = 0
# for filename in os.listdir(folder):
# #     print(filename)
#     if not filename.endswith("json"):
#         continue
    
#     annotation_info = json.load(
#         open(os.path.join(folder, filename), "r", encoding="utf-8")
#     )
# #     print(annotation_info)
    
#     expert_anno_1 = annotation_info.get("expert_labels_1")
#     diagnosis_1 = None
#     if expert_anno_1:
#         diagnosis_1 = expert_anno_1["diagnosis"]
#         if expert_anno_1["quality"] == "no_cough":
#             continue
    
#     expert_anno_2 = annotation_info.get("expert_labels_2")
#     diagnosis_2 = None
#     if expert_anno_2:
#         diagnosis_2 = expert_anno_2["diagnosis"]
#         if expert_anno_2["quality"] == "no_cough":
#             continue
    
#     expert_anno_3 = annotation_info.get("expert_labels_3")
#     diagnosis_3 = None
#     if expert_anno_3:
#         diagnosis_3 = expert_anno_3.get("diagnosis")
# #         diagnosis_3 = expert_anno_3["diagnosis"]
#         if expert_anno_3["quality"] == "no_cough":
#             continue
    
#     if not diagnosis_1 and not diagnosis_2 and not diagnosis_3:
#         continue
    
#     status = annotation_info.get("status")
    
#     diagnose_label = None
#     if set([diagnosis_1, diagnosis_2, diagnosis_3]) == set(["healthy_cough", None]):
#         diagnose_label = "healthy"
#     elif set([diagnosis_1, diagnosis_2, diagnosis_3]) == set(["healthy_cough"]):
#         diagnose_label = "healthy"
#     elif "healthy" not in json.dumps([diagnosis_1, diagnosis_2, diagnosis_3]):
#         diagnose_label = "unhealthy"
#     else:
#         if status:
#             if status == "healthy":
#                 diagnose_label = "healthy"
#             else:
#                 diagnose_label = "unhealthy"
        
#         else:        
#             print(json.dumps(sorted([diagnosis_1, diagnosis_2, diagnosis_3])))
#             if "\"healthy_cough\", \"healthy_cough\"" not in json.dumps(sorted([diagnosis_1, diagnosis_2, diagnosis_3])):
#                 diagnose_label = "unhealthy"
#             else:
#                 diagnose_label = "healthy"
    
#     count += 1
#     dict_id2label[filename.replace(".json", "")] = 1 if diagnose_label == "unhealthy" else 0

# print("有效样本： ", count)


In [4]:
# # 正负样本数量：
# neg_sample = 0   # healthy coughs
# for k, v in dict_id2label.items():
#     if v == 0:
#         neg_sample += 1

# print("负样本率： ", neg_sample / count)

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [5]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}_mean'
for i in range(1, 21):
    header += f' mfcc{i}_std'
header += ' label'


## Writing data to csv file

We write the data to a csv file 

In [6]:
# f = open('./datasets/coughvid_v1/features/data.csv', 'w', encoding="utf-8")
# f.write(header + "\n")

# from tqdm import tqdm
# for filename in tqdm(os.listdir(folder)):
    
#     if filename.endswith("json"):
#         continue
#     if filename.endswith("csv"):
#         continue
    
#     idx = filename.split(".")[0]
#     if idx not in dict_id2label:
#         continue
    
#     label = dict_id2label[idx]
    
#     # print(label, idx)
    
#     # load file
#     y, sr = librosa.load(os.path.join(folder, filename), mono=True, duration=30)
#     # print(y.shape)
#     # print(sr)
    
#     chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
#     rmse = librosa.feature.rms(y=y, )
#     spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
#     spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
#     rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
#     zcr = librosa.feature.zero_crossing_rate(y)
#     mfcc = librosa.feature.mfcc(y=y, sr=sr)
#     to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    

#     for e in mfcc:
#         to_append += f' {np.mean(e)}'
#     for e in mfcc:
#         to_append += f' {np.std(e)}'
            
#     to_append += f' {label}'
#     # print(to_append)
#     f.write(to_append + "\n")

# f.close()

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [7]:
data = pd.read_csv('./datasets/coughvid_v1/features/data.csv', sep=" ")
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1_mean,mfcc2_mean,mfcc3_mean,...,mfcc12_std,mfcc13_std,mfcc14_std,mfcc15_std,mfcc16_std,mfcc17_std,mfcc18_std,mfcc19_std,mfcc20_std,label
0,22dcfcdd-52a8-4b42-9c50-c0a3696382a9.webm,0.187894,0.025356,963.070505,846.074789,1932.604980,0.059711,-472.439301,16.604473,-7.377993,...,11.093034,8.206923,8.646673,6.249838,3.590692,3.762098,9.720296,3.636496,4.808792,1
1,d50fb043-08aa-4960-94f7-e9bbc690e70d.webm,0.042411,0.006042,234.919076,205.140347,446.992558,0.015784,-562.550110,12.425411,-3.398605,...,7.752041,5.287385,4.998383,4.661725,7.737397,5.847256,6.207181,5.978804,2.865030,1
2,5f5f4027-45d4-4867-a9e2-ecc500eaf21d.webm,0.557196,0.015863,2785.087057,2211.925156,5268.870618,0.191680,-505.479309,31.321623,-5.810165,...,7.691325,6.780107,4.998759,4.860253,7.856476,3.556230,4.408528,4.954839,8.589847,1
3,8a5feabb-3b0a-49b5-900c-6d3d31e96794.webm,0.427177,0.054161,2766.423560,2258.583563,5397.398633,0.235709,-333.071655,48.053001,-18.119431,...,7.634245,8.813775,7.188757,8.187786,8.326145,7.226190,7.231071,8.264647,6.015858,1
4,960336ac-2567-4136-8ee8-f8564d556822.webm,0.421020,0.031633,2478.157650,1981.974390,4688.709486,0.219031,-455.479889,35.985081,-29.793978,...,13.291973,9.249558,8.176953,6.090113,7.761150,6.474304,5.309581,5.311271,8.227166,1


In [8]:
data.shape

(2138, 48)

In [9]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [ ]:

from src.baseline1_feature_extraction.tree_training_utils import train_cv_with_thres, threshold_prob, threshold_prob_via_jordan

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

random_state = 100
cbt = CatBoostClassifier(verbose=0, allow_writing_files=False, random_state=random_state)
lgb = LGBMClassifier(is_unbalance=True, random_state=random_state)
xgb = XGBClassifier(verbose=0, scale_pos_weight=3, enable_categorical=True, random_state=random_state)
logistic = LogisticRegression(verbose=0, random_state=random_state)


In [ ]:
# models 
list_model_names = [
    "cbt",
    "lgb",
    "xgb",
    "logistic",
#     "stack"
]
list_model_clfs = [
    cbt, 
    lgb, 
    xgb, 
    logistic, 
#     stack
]

In [ ]:
import json

features = [w for w in data.columns if w not in ["label"]]
print(features)

for clf_name, clf in zip(list_model_names, list_model_clfs):
    
    res = train_cv_with_thres(data,
                        clf,
                        features,
                        num_folds=5,
                        random_state=101,
                        sensitivity_threshold=0.75,
                        save_dir=None,
                        )
    print(f"**{clf_name}**" * 30)
    print(res)
    print(f"**{clf_name}**" * 30)

## Encoding the Labels

In [10]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [11]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
len(y_train)

1710

In [14]:
len(y_test)

428

In [15]:
X_train[10]

array([ 0.3591301 , -0.25922497, -0.57788119,  0.06884817, -0.33089911,
       -0.83752411,  0.32502235,  0.90580188, -0.13074752, -0.27530241,
        0.62560535, -1.17995162,  1.26249642,  0.40421693,  0.3379524 ,
        1.23840052,  1.66424156,  1.42644844,  0.73458758, -0.07949798,
       -0.609472  ,  0.26316202, -0.32750201, -1.96656077, -1.47049683,
       -0.28576964, -0.19126953,  1.66445639,  0.33872014, -0.94282109,
       -1.3252734 , -0.98665131, -1.77422902, -1.48956221, -0.58558322,
       -1.67459028, -1.42597955, -2.01056192, -1.48048068, -0.86154918,
       -1.06210475, -1.12256077, -1.2109277 ,  0.58490895, -0.2545372 ,
       -1.34971647])

## cls with CatBoost


1it [00:05,  5.55s/it]

(428, 1)
(428, 1)
(428, 2)
(428, 1)
(428, 2)


2it [00:11,  5.77s/it]

(428, 1)
(428, 1)
(428, 2)
(428, 1)
(428, 2)


3it [00:16,  5.57s/it]

...

(427, 1)
(427, 1)
(427, 2)
(427, 1)
(427, 2)
[11:42:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "enable_categorical", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:42:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


5it [00:03,  1.51it/s]


(427, 1)
(427, 1)
(427, 2)
(427, 1)
(427, 2)
**xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb****xgb**
{'敏感性': [0.752808988764045, 0.752808988764045, 0.752808988764045, 0.75, 0.752808988764045], '特异性': [0.39823008849557523, 0.31268436578171094, 0.3421828908554572, 0.36283185840707965, 0.4319526627218935], '敏感性_jordan': [0.7303370786516854, 0.2134831460674157, 0.2808988764044944, 0.6704545454545454, 0.6179775280898876], '特异性_jordan': [0.45132743362831856, 0.9144542772861357, 0.8908554572271387, 0.5191740412979351, 0.5828402366863905], '阈值': [0.9904885, 0.99485695, 0.9931617, 0.9930642, 0.99070144], '阈值_jordan': [0.9883739, 0.8505618, 0.8173863, 0.9850188, 0.984303], 'auc': [0.6050843525239469, 0.5623943521925028, 0.5872195154287229, 0.6134687583802628, 0.616315404560867], 'acc': [0.5070093457943925, 0.7663551401869159, 0.761682242990654

5it [00:00, 37.59it/s]

(428, 1)
(428, 1)
(428, 2)
(428, 1)
(428, 2)
(428, 1)
(428, 1)
(428, 2)
(428, 1)
(428, 2)
(428, 1)
(428, 1)
(428, 2)
(428, 1)
(428, 2)
(427, 1)
(427, 1)
(427, 2)
(427, 1)
(427, 2)
(427, 1)
(427, 1)
(427, 2)
(427, 1)
(427, 2)
**logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic****logistic**
{'敏感性': [0.7640449438202247, 0.7752808988764045, 0.7640449438202247, 0.75, 0.7640449438202247], '特异性': [0.3687315634218289, 0.4424778761061947, 0.45427728613569324, 0.45427728613569324, 0.44970414201183434], '敏感性_jordan': [0.6741573033707865, 0.7752808988764045, 0.8202247191011236, 0.7954545454545454, 0.696629213483146], '特异性_jordan': [0.5663716814159292, 0.4424778761061947, 0.41002949852507375, 0.4159292035398

# Classification with Keras

## Building our Network

In [20]:
from keras import models
from keras import layers
from keras import callbacks

model = models.Sequential()
model.add(layers.Dense(32, activation='gelu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(32, activation='gelu'))
model.add(layers.Dense(32, activation='gelu'))
model.add(layers.Dense(10, activation='softmax'))

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    epochs=30,
                    batch_size=8,
                    callbacks=callbacks.EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.005,
                        patience=5,
                        verbose=0,
                        mode='auto',
                        baseline=None,
                        restore_best_weights=True
                    )
                   )
                   

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

In [ ]:
print('test_acc: ',test_acc)

Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [ ]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [ ]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

In [ ]:
results

## Predictions on Test Data

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])